The ImagePromptViewModel handles user interactions and image data presentation by coordinating with the Unsplash service, following the MVVM pattern for clean separation of logic and UI.

- _unsplashService: handles API calls to fetch images from Unsplash- 
- DefaultImageCount and AdditionalImages: are constants defining the initial loading state and on demand
- isLoading: tracks whether an image oepration is in progress
- _undoStack: stores deleted images and their position for undo functionality
- Images: observable collection of ImageItems objects bound to the UI

In [ ]:
private readonly Unsplash _unsplashService;
private const int DefaultImageCount = 3;
private const int AdditionalImages = 1;
private bool _isLoading;

private readonly Stack<(ImageItem item, int index)> _undoStack = new();

public ObservableCollection<ImageItem> Images { get; } = new();

The boolean properties below manages UI state and interactivity
<break>

- IsLoading: whether an image operation is in progress and notifies OUI of state changes and updates command avaliability.
- CanUndo: returns true if there are deleted inmages in the undo stack to enable the Undo command.

In [ ]:
public bool IsLoading
{
    get => _isLoading;
    set
    {
        if (_isLoading != value)
        {
            _isLoading = value;
            OnPropertyChanged();
            CommandsCanExecuteChanged();
        }
    }
}

public bool CanUndo => _undoStack.Count > 0;

[ICommand](https://learn.microsoft.com/en-us/dotnet/api/system.windows.input.icommand?view=net-9.0)s connect UI actions (buttons) to code method. 

In [ ]:
public ICommand LoadInitialImagesCommand { get; }
public ICommand AddImagesCommand { get; }
public ICommand RegenerateImagesCommand { get; }
public ICommand ToggleLockCommand { get; }
public ICommand DeleteImageCommand { get; }
public ICommand UndoDeleteCommand { get; }

Constructor:
- Initializes the ImagePromptViewModwl by setting up the command bindings for image operations (Load, Add, Regenerate, Lock, Delete, Undo).
- The last line ensures that initial images are loaded on teh main thread, guaranteeing UI responsiveness during image fetching.

In [ ]:
public ImagePromptViewModel()
{
    _unsplashService = new Unsplash();

    LoadInitialImagesCommand = new Command(async () => await LoadInitialImagesAsync(), () => !IsLoading);
    AddImagesCommand = new Command(async () => await AddImagesAsync(), () => !IsLoading);
    RegenerateImagesCommand = new Command(async () => await RegenerateImagesAsync(), () => !IsLoading);
    ToggleLockCommand = new Command<ImageItem>(ToggleLock);
    DeleteImageCommand = new Command<ImageItem>(DeleteImage);
    UndoDeleteCommand = new Command(UndoDelete, () => CanUndo);

    // Load initial images on main thread
    MainThread.BeginInvokeOnMainThread(async () => await LoadInitialImagesAsync());
}

Command Logic:
<break>

This section defines methods for managing image operations within the class (loading initial images, regenerating images, toggling lock state, undoing deletion). These methods helps control the state and functionality of the UI.
<break>

Example in LoadInitialImageAsync(), this method fetches a predefined number of random images from the API, updates the observable collection, and notifies the UI while handling any potential erros.

In [ ]:
private async Task LoadInitialImagesAsync()
{
    try
    {
        IsLoading = true;
        Images.Clear();

        var images = await _unsplashService.GetRandomImagesAsync(DefaultImageCount);
        foreach (var image in images)
        {
            Images.Add(new ImageItem(image));
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Error fetching images: {ex.Message}");
    }
    finally
    {
        IsLoading = false;
    }
}

private async Task RegenerateImagesAsync() {}

private void ToggleLock(ImageItem? imageItem){}

private void UndoDelete() {}


This method implements a Snackbar that notifies the user when an image is deleted or undone, displaying the notification for 5 seconds before it disappears.

In [ ]:
private async void DeleteImage(ImageItem? imageItem)
{
    if (imageItem != null && Images.Contains(imageItem))
    {
        int index = Images.IndexOf(imageItem);
        _undoStack.Push((imageItem, index));
        Images.RemoveAt(index);

        OnPropertyChanged(nameof(CanUndo));
        ((Command)UndoDeleteCommand).ChangeCanExecute();

        var snackbar = Snackbar.Make(
            "Image deleted",
            () => UndoDelete(),
            "Undo",
            TimeSpan.FromSeconds(5));

        await snackbar.Show();
    }
}

This method updates the executable state of the image-related commands and notifies the UI of potential changes to the undo functionality.

In [ ]:
private void CommandsCanExecuteChanged()
{
    ((Command)AddImagesCommand).ChangeCanExecute();
    ((Command)RegenerateImagesCommand).ChangeCanExecute();
    ((Command)UndoDeleteCommand).ChangeCanExecute();
    OnPropertyChanged(nameof(CanUndo));
}

This section defines the ImageItem class, which controls the state of UI elements (such as lock and delete buttons) based on the IsLocked and IsDeleted properties. It also determines the appropriate icon for the delete/undo button.

In [ ]:
// A snipper:
public class ImageItem : INotifyPropertyChanged
{
    private bool _isLocked;
    private bool _isDeleted;

    public UnsplashImage UnsplashImage { get; }

    public bool IsLocked
    {
        get => _isLocked;
        set
        {
            if (_isLocked != value)
            {
                _isLocked = value;
                OnPropertyChanged();
            }
        }
    }
    // ETC
}

Interface: [INotifyPropertyChange](https://learn.microsoft.com/en-us/dotnet/api/system.runtime.compilerservices.callermembernameattribute?view=net-9.0) implementation using OnPropertyChanged method with CallerMemberName attribute:

"Implementing the INotifyPropertyChanged interface when binding data. This interface allows the property of an object to notify a bound control that the property has changed, so that the control can display the updated information."

In [ ]:
public event PropertyChangedEventHandler? PropertyChanged;

protected virtual void OnPropertyChanged([CallerMemberName] string? propertyName = null)
{
    PropertyChanged?.Invoke(this, new PropertyChangedEventArgs(propertyName));
}